In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.nn.modules.activation import LeakyReLU

In [3]:
synthetic_calls_path = '/content/drive/MyDrive/Progetto Stage/data/binom_synthetic_calls.csv'
synthetic_puts_path = '/content/drive/MyDrive/Progetto Stage/data/binom_synthetic_puts.csv'

In [4]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
    
    return df

In [5]:
synthetic_calls = pd.read_csv(synthetic_calls_path, index_col=0)
synthetic_puts = pd.read_csv(synthetic_puts_path, index_col=0)

synthetic_calls = reduce_mem_usage(synthetic_calls)
synthetic_puts = reduce_mem_usage(synthetic_puts)

In [6]:
synthetic_options = pd.concat([synthetic_calls, synthetic_puts], axis=0)
synthetic_options = shuffle(synthetic_options)
synthetic_options = synthetic_options.reset_index()
synthetic_options = synthetic_options.drop('index', axis=1)

In [7]:
synthetic_options.head(5)

,Price,Strike,Type,Vol,Interest Rate,Time to Expiration,Option Price
0,100,126.0,C,0.850098,0.099976,0.300049,11.140625
1,100,132.0,P,0.600098,0.049988,0.600098,37.843750
2,100,160.0,P,0.399902,0.090027,1.000000,50.593750
3,100,67.0,P,0.549805,0.010002,0.600098,3.121094
4,100,160.0,C,0.649902,0.049988,0.600098,6.492188


# Preprocessing

In [8]:
synthetic_options = pd.get_dummies(synthetic_options, prefix='', prefix_sep='')

In [9]:
input_sc = StandardScaler()
output_sc = StandardScaler()
input_data = input_sc.fit_transform(synthetic_options.drop('Option Price', axis=1))
output_data = output_sc.fit_transform(synthetic_options['Option Price'].values.reshape(-1, 1))

train_size = 0.8
val_size = 0.1

last_train_idx = int(np.round(len(input_data) * train_size))
last_val_idx = last_train_idx + int(np.round(len(input_data) * val_size))

X_train = input_data[0:last_train_idx]
X_val = input_data[last_train_idx:last_val_idx]
X_test = input_data[last_val_idx:]

y_train = output_data[0:last_train_idx]
y_val = output_data[last_train_idx:last_val_idx]
y_test = output_data[last_val_idx:]

In [10]:
X_train = Variable(torch.Tensor(X_train))
X_val = Variable(torch.Tensor(X_val))
X_test = Variable(torch.Tensor(X_test))

y_train = Variable(torch.Tensor(y_train))
y_val = Variable(torch.Tensor(y_val))
y_test = Variable(torch.Tensor(y_test))

 # Model

In [11]:
CUDA = torch.cuda.is_available()
device = 'cuda:0' if CUDA else 'cpu'

In [12]:
class HiddenLayer(nn.Module):

  def __init__(self, layer_size, dropout=0.1):
      super(HiddenLayer, self).__init__()
      self.layer = nn.Sequential(
          nn.Linear(layer_size, layer_size),
          nn.LeakyReLU(),
          nn.Dropout(dropout)
      )
    
  def forward(self, x):
    return self.layer(x)

# Model 1

- 4 hidden layers
- Leaky ReLU as activation function

In [13]:
class Net(nn.Module):

  def __init__(self, input_size, output_size, hidden_size):
    super(Net, self).__init__()
    self.input_size = input_size
    self.output_size = output_size
    self.hidden_size = hidden_size

    self.net = nn.Sequential(
        nn.Linear(self.input_size, self.hidden_size),
        nn.LeakyReLU(),
        HiddenLayer(self.hidden_size),
        HiddenLayer(self.hidden_size),
        HiddenLayer(self.hidden_size),
        HiddenLayer(self.hidden_size),  
        nn.Linear(self.hidden_size, self.output_size)
    )
  
  def forward(self, x):
    return self.net(x)

# Model 2

- Number of Layer as input
- same as model 1

code: https://stackoverflow.com/questions/62937388/pytorch-dynamic-amount-of-layers

In [14]:
# TODO: to implement

# Model 3

- as model 2 but with residual connections

code:  https://stackoverflow.com/questions/57229054/how-to-implement-my-own-resnet-with-torch-nn-sequential-in-pytorc

In [15]:
#TODO: to implement

# Training

In [16]:
input_size = 7
hidden_size = 64
output_size = 1
batch_size = 1208
epochs = 1000
lr = 1e-4

model = Net(input_size, output_size, hidden_size)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [17]:
model = model.to(device)

X_train = X_train.to(device)
y_train = y_train.to(device)

X_val = X_val.to(device)
y_val = y_val.to(device)

X_test = X_test.to(device)
y_test = y_test.to(device)

In [18]:
class OptDataset(Dataset):

  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

  def __len__(self):
    return len(self.X)

In [19]:
def evaluate(model, loss_fn, X_val, y_val):
  model.eval()
  with torch.no_grad():
    out = model(X_test)
    loss = loss_fn(out, y_test)
    print('\nVal set: Average loss: {:.8f}\n'.format(
            loss.item()))
    return loss.item()

In [20]:
def train(
    epochs, 
    batch_size, 
    model,
    optimizer,
    loss_fn, 
    X_train,
    y_train,
    X_val,
    y_val
):
  for epoch in range(epochs):
    model.train()
    total_loss = 0
    start_time = time.time()
    i = 0

    for batch, batch_labels in DataLoader(OptDataset(X_train, y_train), batch_size=batch_size):
      out = model(batch.to(device))
      optimizer.zero_grad()

      loss = loss_fn(out, batch_labels.to(device))
      total_loss += loss.item()
      loss.backward()
      optimizer.step()

      if i > 0 and i % 50 == 0:
        avg_loss = total_loss / 50
        elapsed = time.time() - start_time
        print('| Epoch {:3d} | {:5d}/{:5d} batches | lr {:2.5f} | ms/batch {:5.2f} | '
                  'loss {:5.8f}'.format(
              epoch, i, len(X_train) // batch_size+1, lr, elapsed * 1000 / 50,
              avg_loss))
        start_time = time.time()
        total_loss = 0

      i += 1
    
    evaluate(model, loss_fn, X_val, y_val)

In [ ]:
load = False
save_model_path = f'/content/drive/MyDrive/Progetto Stage/models/opt_net1_h{hidden_size}.chkpt'

if not load:
  train(
      epochs, 
      batch_size, 
      model, 
      optimizer, 
      loss_fn, 
      X_train, 
      y_train,
      X_val,
      y_val)
  torch.save(model.state_dict(), save_model_path)
else:
  model = Net(input_size, output_size, hidden_size)
  model.load_state_dict(torch.load(save_model_path, map_location=device))

| Epoch   0 |    50/  401 batches | lr 0.00010 | ms/batch 15.46 | loss 1.01290744
| Epoch   0 |   100/  401 batches | lr 0.00010 | ms/batch 10.89 | loss 0.92099674
| Epoch   0 |   150/  401 batches | lr 0.00010 | ms/batch 11.20 | loss 0.68641208
| Epoch   0 |   200/  401 batches | lr 0.00010 | ms/batch 12.39 | loss 0.31637840
| Epoch   0 |   250/  401 batches | lr 0.00010 | ms/batch 11.35 | loss 0.16292509
| Epoch   0 |   300/  401 batches | lr 0.00010 | ms/batch 10.84 | loss 0.10498055
| Epoch   0 |   350/  401 batches | lr 0.00010 | ms/batch 11.05 | loss 0.07734603

Val set: Average loss: 0.01922312

| Epoch   1 |    50/  401 batches | lr 0.00010 | ms/batch 11.27 | loss 0.05400861
| Epoch   1 |   100/  401 batches | lr 0.00010 | ms/batch 11.74 | loss 0.04641830
| Epoch   1 |   150/  401 batches | lr 0.00010 | ms/batch 12.56 | loss 0.04169635
| Epoch   1 |   200/  401 batches | lr 0.00010 | ms/batch 11.52 | loss 0.03928902
| Epoch   1 |   250/  401 batches | lr 0.00010 | ms/batch 10.9

# Test the model

In [ ]:
test_out = model(X_test[0:10])
test_out = output_sc.inverse_transform(test_out.cpu().detach().numpy())
test_out

In [ ]:
output_sc.inverse_transform(y_test[0:10].cpu().detach().numpy())